<a href="https://colab.research.google.com/github/Binamraaa/climate-near-future/blob/main/Climate_Model_Future_Days.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Interactive Simple Climate Model Simulation (using FigureWidget)

This notebook simulates near-term climate change based on key parameters using a simple time-stepping model. It explores two scenarios:

1.  **Business-as-Usual (BAU):** Continued exponential CO2 growth and associated aerosol effects (cooling effect capped after 2015).
2.  **World Without Us (WWU):** An abrupt cessation of emissions and aerosols starting when BAU CO2 reaches 400 ppm. Aerosols disappear instantly, while CO2 levels slowly decay towards a new equilibrium (340 ppm).

Use the sliders below the code cells to interactively explore the impact of major climate uncertainties on the projected CO2 concentration, radiative forcing, and temperature anomaly.

In [ ]:
import numpy as np
import math
import pandas as pd

# Import Plotly & Set Renderer FOR COLAB
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
pio.renderers.default = 'colab' # <<< ENSURE THIS IS SET FOR COLAB

# Import ipywidgets
import ipywidgets as widgets
from IPython.display import display, clear_output # Import clear_output

print("Libraries imported and Plotly renderer set for Colab.")

from google.colab import output
output.enable_custom_widget_manager()

Libraries imported and Plotly renderer set for Colab.


## Model Parameters and Constants

Defines the core physical constants, initial conditions, simulation time steps, and scenario parameters.

In [ ]:
# --- Simulation Parameters ---
D_time = 1.0            # Time step (years)
eqCO2 = 280.0           # Equilibrium pre-industrial CO2 (ppm)
initialCO2 = 290.0      # Starting CO2 for simulation in 1900 (ppm)
start_year = 1900
end_year = 2100
present_day_year = 2015 # Year for aerosol target and capping

# --- CO2 Change Rates ---
co2_growth_rate_A = 0.0225 / D_time # Exponential growth rate [/year]
co2_drawdown_rate = 0.01 / D_time   # Exponential drawdown rate [/year] in WWU

# --- Radiative Forcing & Response ---
rf_co2_doubling_4Wm2 = 4.0 # Radiative forcing for doubling CO2 (W/m^2)
t_response_time = 20.0     # Temperature relaxation timescale (years)

# --- WWU Scenario Specifics ---
wwu_target_co2 = 340.0    # Target CO2 for WWU drawdown (ppm)
wwu_trigger_co2 = 400.0   # CO2 level to trigger WWU scenario (ppm)

print("Parameters set.")

Parameters set.


## Core Simulation Function

This function encapsulates the main time-stepping calculation logic. It takes climate sensitivity and present-day aerosol forcing as inputs and calculates the yearly evolution of CO2, radiative forcings, and temperatures for both the BAU and WWU scenarios.

In [ ]:
def run_climate_simulation(climateSensitivity2x, aerosolRfToday_target):
    """Runs the climate model simulation with given parameters."""
    # Calculate derived parameters
    if rf_co2_doubling_4Wm2 > 0:
        climateSensitivityWm2 = climateSensitivity2x / rf_co2_doubling_4Wm2
    else:
        climateSensitivityWm2 = 0 # Avoid division by zero

    # --- Create Lists ---
    years = list(range(start_year, end_year + 1))
    num_years = len(years)

    bauCO2 = [0.0] * num_years
    bauCO2RateOfChange = [0.0] * num_years
    bauRfCO2 = [0.0] * num_years
    bauRfMasking = [0.0] * num_years
    bauRfTotal = [0.0] * num_years
    bauTeq = [0.0] * num_years
    bauTtransient = [0.0] * num_years

    wwuCO2 = [np.nan] * num_years # Initialize with NaN for plotly gaps
    wwuRfCO2 = [np.nan] * num_years
    wwuRfTotal = [np.nan] * num_years
    wwuTeq = [np.nan] * num_years
    wwuTtransient = [np.nan] * num_years

    # --- First Loop: BAU CO2 Growth ---
    bauCO2[0] = initialCO2
    bauCO2RateOfChange[0] = 0.0
    for i in range(1, num_years):
        prev_co2 = bauCO2[i-1]
        current_co2 = eqCO2 + (prev_co2 - eqCO2) * (1 + co2_growth_rate_A * D_time)
        bauCO2[i] = current_co2
        rate_change = (current_co2 - prev_co2) / D_time
        bauCO2RateOfChange[i] = rate_change

    # --- Calculate B (Aerosol Scaling Factor) ---
    masking_param_B = 0
    present_day_index = -1
    try:
        present_day_index = years.index(present_day_year)
        co2_rate_at_present = bauCO2RateOfChange[present_day_index]
        if co2_rate_at_present != 0:
            masking_param_B = aerosolRfToday_target / co2_rate_at_present
    except ValueError:
        pass # Keep masking_param_B as 0


    # --- Second Loop: BAU Forcings & Temperature ---
    rf_masking_at_2015_calculated = masking_param_B * co2_rate_at_present if present_day_index !=-1 and co2_rate_at_present != 0 else 0
    bauTtransient[0] = 0.0 # Assume equilibrium at start_year
    for i in range(1, num_years):
        # RF CO2
        if bauCO2[i] > 0 and eqCO2 > 0:
            bauRfCO2[i] = rf_co2_doubling_4Wm2 * math.log(bauCO2[i] / eqCO2) / math.log(2)
        else: bauRfCO2[i] = -float('inf')

        # RF Masking (scaled and capped)
        rf_masking_scaled = masking_param_B * bauCO2RateOfChange[i]
        if years[i] >= present_day_year:
             bauRfMasking[i] = max(rf_masking_scaled, rf_masking_at_2015_calculated)
        else: bauRfMasking[i] = rf_masking_scaled

        # Total RF & Equilibrium Temperature
        bauRfTotal[i] = bauRfCO2[i] + bauRfMasking[i]
        bauTeq[i] = bauRfTotal[i] * climateSensitivityWm2

        # Transient Temperature
        prev_Ttransient = bauTtransient[i-1]
        delta_T = (bauTeq[i] - prev_Ttransient) / t_response_time * D_time
        bauTtransient[i] = prev_Ttransient + delta_T

    # --- WWU Scenario Stage ---
    i_wwu_start = -1
    wwu_start_year = -1
    for i in range(num_years):
        if bauCO2[i] >= wwu_trigger_co2:
            i_wwu_start = i
            wwu_start_year = years[i]
            break

    if i_wwu_start != -1:
        # Initialize WWU at divergence point
        wwuCO2[i_wwu_start] = bauCO2[i_wwu_start]
        wwuTtransient[i_wwu_start] = bauTtransient[i_wwu_start]

        # Calculate initial WWU RF/Teq (CO2 only, no masking)
        current_wwu_co2 = wwuCO2[i_wwu_start]
        if current_wwu_co2 > 0 and eqCO2 > 0:
            current_wwu_rfco2 = rf_co2_doubling_4Wm2 * math.log(current_wwu_co2 / eqCO2) / math.log(2)
        else: current_wwu_rfco2 = -float('inf')
        wwuRfCO2[i_wwu_start] = current_wwu_rfco2
        wwuRfTotal[i_wwu_start] = current_wwu_rfco2
        wwuTeq[i_wwu_start] = current_wwu_rfco2 * climateSensitivityWm2

        # Loop for subsequent WWU years
        for i in range(i_wwu_start + 1, num_years):
            prev_wwu_co2 = wwuCO2[i-1] # Assumes previous step was calculated
            delta_co2 = (wwu_target_co2 - prev_wwu_co2) * co2_drawdown_rate * D_time
            current_wwu_co2 = prev_wwu_co2 + delta_co2
            wwuCO2[i] = current_wwu_co2

            # RF CO2 & Total RF (no masking)
            if current_wwu_co2 > 0 and eqCO2 > 0:
                 current_wwu_rfco2 = rf_co2_doubling_4Wm2 * math.log(current_wwu_co2 / eqCO2) / math.log(2)
            else: current_wwu_rfco2 = -float('inf')
            wwuRfCO2[i] = current_wwu_rfco2
            wwuRfTotal[i] = current_wwu_rfco2

            # Equilibrium & Transient Temperature
            wwuTeq[i] = current_wwu_rfco2 * climateSensitivityWm2
            prev_wwu_Ttransient = wwuTtransient[i-1] # Assumes previous step was calculated
            delta_T = (wwuTeq[i] - prev_wwu_Ttransient) / t_response_time * D_time
            wwuTtransient[i] = prev_wwu_Ttransient + delta_T

    # Return results dictionary
    return {
        'years': years,
        'bauCO2': bauCO2, 'bauRfCO2': bauRfCO2, 'bauRfMasking': bauRfMasking,
        'bauRfTotal': bauRfTotal, 'bauTtransient': bauTtransient,
        'wwuCO2': wwuCO2, 'wwuRfTotal': wwuRfTotal, 'wwuTtransient': wwuTtransient,
        'i_wwu_start': i_wwu_start, 'wwu_start_year': wwu_start_year,
        'params': {'sens': climateSensitivity2x, 'aero': aerosolRfToday_target}
    }

print("Simulation function defined.")

Simulation function defined.


## Plotting Function

This function uses Plotly to generate the three subplots based on the results dictionary. It creates a new `go.Figure` each time it's called. Includes annotation font fixes.

In [ ]:
def plot_climate_results_plotly(results):
    """Plots the 3 graphs using Plotly with layout adjustments."""
    years = results['years']
    i_wwu_start = results['i_wwu_start']
    wwu_start_year = results['wwu_start_year']
    climateSensitivity2x = results['params']['sens']
    aerosolRfToday_target = results['params']['aero']

    # Define annotation texts
    hline_text = f'Trigger ({wwu_trigger_co2:.0f} ppm)'
    vline_text = "" # Default empty
    if i_wwu_start != -1:
        vline_text = f'WWU Start ({wwu_start_year})'

    # Create subplots - Use go.Figure
    fig = make_subplots(
        rows=3, cols=1,
        shared_xaxes=True,
        vertical_spacing= 0.05, # <<< Increased further
        subplot_titles=( # <<< Use shorter titles here
            "CO₂ Conc",
            "RF",
            "TA"
        )
    )

    # --- Plot 1: pCO2 ---
    fig.add_trace(go.Scatter(x=years, y=results['bauCO2'], name='BAU pCO2',
                             mode='lines', line=dict(color='red'), legendgroup="co2"), row=1, col=1)
    if i_wwu_start != -1:
         fig.add_trace(go.Scatter(x=years, y=results['wwuCO2'], name='WWU pCO2',
                                  mode='lines', line=dict(color='blue', dash='dash'), legendgroup="co2"), row=1, col=1)
    fig.add_hline(y=wwu_trigger_co2, line=dict(color='lightgrey', dash='dot', width=1),
                  annotation_text=hline_text, annotation_position="bottom left", row=1, col=1)

    # --- Plot 2: Radiative Forcing ---
    fig.add_trace(go.Scatter(x=years, y=results['bauRfCO2'], name='BAU RF CO2',
                             mode='lines', line=dict(color='salmon'), legendgroup="rf"), row=2, col=1)
    fig.add_trace(go.Scatter(x=years, y=results['bauRfMasking'], name=f'BAU RF Masking',
                             mode='lines', line=dict(color='dodgerblue', width=2), legendgroup="rf"), row=2, col=1)
    fig.add_trace(go.Scatter(x=years, y=results['bauRfTotal'], name='BAU RF Total',
                             mode='lines', line=dict(color='black', width=2), legendgroup="rf"), row=2, col=1)
    if i_wwu_start != -1:
        fig.add_trace(go.Scatter(x=years, y=results['wwuRfTotal'], name='WWU RF Total',
                                 mode='lines', line=dict(color='green', dash='dash', width=2), legendgroup="rf"), row=2, col=1)

    # --- Plot 3: Temperature Anomaly ---
    fig.add_trace(go.Scatter(x=years, y=results['bauTtransient'], name='BAU Temp Anomaly',
                             mode='lines', line=dict(color='red'), legendgroup="temp"), row=3, col=1)
    if i_wwu_start != -1:
        fig.add_trace(go.Scatter(x=years, y=results['wwuTtransient'], name='WWU Temp Anomaly',
                                 mode='lines', line=dict(color='blue', dash='dash'), legendgroup="temp"), row=3, col=1)

    # Add vertical line if WWU starts
    if i_wwu_start != -1:
        fig.add_vline(x=wwu_start_year, line=dict(color='grey', dash='dot', width=1),
                      annotation_text=vline_text, annotation_position="bottom right", row='all', col=1)

    # --- Modify specific annotation font sizes ---
    new_font_size = 7
    if fig.layout.annotations:
        for ann in fig.layout.annotations:
            if ann.text == hline_text or ann.text == vline_text:
                 ann.font = dict(size=new_font_size, color="grey")

    # --- Layout Updates ---
    fig.update_layout(
        height=900,
        hovermode='x unified',
        legend=dict( # <<< More positioning options
            orientation="h",
            yanchor="bottom",
            y=1.2,
            xanchor="center", # <<< Center legend
            x=0.5
        ),
        margin=dict(t=120, b=50, l=60, r=30), # <<< Adjusted margins (esp. top)
        # Set main title within layout margin
        title=dict(
            text=f"Simulated Climate Scenarios (ΔT_2x={climateSensitivity2x:.1f}°C, Aerosol RF={aerosolRfToday_target:.2f} W/m²)",
            y=0.85, # Position title high within top margin
            x=0.5,
            xanchor='auto',
            yanchor='auto'
        ),
        template="plotly_white"
    )
    # Update axes titles
    fig.update_yaxes(title_text="pCO2 (ppm)", row=1, col=1, title_standoff=10) # Increased standoff
    fig.update_yaxes(title_text="Forcing (W/m²)", zeroline=True, zerolinewidth=1, zerolinecolor='grey', row=2, col=1, title_standoff=10)
    fig.update_yaxes(title_text="Temp Anomaly (°C)", zeroline=True, zerolinewidth=1, zerolinecolor='grey', row=3, col=1, title_standoff=10)
    fig.update_xaxes(title_text="Year", row=3, col=1)

    return fig

print("Plotting function defined.")

Plotting function defined.


## Interactive Controls and Plot Area

Define the interactive widgets (sliders), a button to trigger plotting, and an Output widget to display the plot.

In [ ]:
# --- Create Interactive Widgets ---
sens_slider = widgets.FloatSlider(
    value=3.0, min=1.0, max=6.0, step=0.1, description='ΔT_2x (°C):',
    continuous_update=False, readout=True, readout_format='.1f',
    style={'description_width': 'initial'}, layout=widgets.Layout(width='60%') # Wider slider
)
aero_slider = widgets.FloatSlider(
    value=-0.75, min=-2.5, max=0.0, step=0.05, description='Aerosol RF (W/m²):',
    continuous_update=False, readout=True, readout_format='.2f',
    style={'description_width': 'initial'}, layout=widgets.Layout(width='60%') # Wider slider
)

# --- Create Update Button ---
update_button = widgets.Button(
    description='Generate Plot',
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to generate plot with current slider values',
    icon='check' # Example icon
)

# --- Create an EXPLICIT Output widget to hold the plot ---
plot_output = widgets.Output()

print("Controls and plot output area created.")

Controls and plot output area created.


## Button Click Handler and Display

Define the function that runs when the button is clicked. This function reads the slider values, runs the simulation, generates the plot, and displays it in the dedicated Output area below. Please consider ignoring the first generated plot. This is apparently how Google Colab handles the dynamic resizing and replacement of complex Javascript outputs like Plotly figures when using ipywidgets and clear_output.

In [ ]:
# --- Define function to run on button click ---
def on_button_clicked(b): # b is the button object passed on click
    """Reads sliders, runs sim, clears output, displays plot in plot_output."""
    with plot_output: # Direct output to the dedicated plot_output widget
        clear_output(wait=True) # Clear previous plot in this specific output area
        simulation_results = run_climate_simulation(
            climateSensitivity2x=sens_slider.value, # Get current slider values
            aerosolRfToday_target=aero_slider.value
        )
        fig = plot_climate_results_plotly(simulation_results)
        fig.show(renderer=pio.renderers.default) # Display plot

# --- Link the button's on_click event to the handler function ---
update_button.on_click(on_button_clicked)

# --- Display the final layout ---
print("Adjust sliders, then click 'Generate Plot':")
controls = widgets.VBox([sens_slider, aero_slider, update_button])
display(controls, plot_output) # Display controls AND the output area

# --- Optional: Generate initial plot on first load ---
# on_button_clicked(None) # Pass None or a dummy object

Adjust sliders, then click 'Generate Plot':


Output()